## Choosing the best checkpoint from the PBT trials

In [1]:
from pathlib import Path

from ray.rllib.agents.ppo import PPOTrainer
from ray.tune import ExperimentAnalysis
from ray.tune.registry import register_env

from inventory_env.env_creator import inventory_env_creator

register_env("inventory_env", inventory_env_creator)

path_to_results_dir = Path().absolute().parent / "inventory_management_rl" / "experiments" / "experiment_results" / "pbt"
analysis = ExperimentAnalysis(path_to_results_dir, default_metric="evaluation/episode_reward_mean", default_mode="max")
best_trial = analysis.get_best_trial(scope="all")
best_checkpoint = analysis.get_best_checkpoint(best_trial)
config = best_trial.config
agent = PPOTrainer(config=config)
agent.restore(best_checkpoint)

2023-08-10 20:35:08,361	WARNING deprecation.py:45 -- DeprecationWarning: `evaluation_num_episodes` has been deprecated. Use ``evaluation_duration` and `evaluation_duration_unit=episodes`` instead. This will raise an error in the future!
2023-08-10 20:35:08,362	WARNING ppo.py:223 -- `train_batch_size` (2048) cannot be achieved with your other settings (num_workers=2 num_envs_per_worker=1 rollout_fragment_length=200)! Auto-adjusting `rollout_fragment_length` to 1024.
(RolloutWorker pid=12347) /home/dibya/miniconda3/envs/real_world_deep_rl_cpu/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=12347)   logger.warn(
(RolloutWorker pid=12345) /home/dibya/miniconda3/envs/real_world_deep_rl_cpu/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=12345)   logger.warn(
2023-08-10 20:35:13,423	WARNING deprecation.py:45 -- Dep

## Measuring average performance using 100000 episodes and same seed

In [2]:
import numpy as np

env = inventory_env_creator(
    {
        "obs_filter": "my_normalize", 
        "reward_filter": None, 
    }, 
    seed=0
)
num_episodes = 100000
all_r = []
for _ in range(num_episodes):
    obs = env.reset()
    ep_r = 0
    while True:
        action = agent.compute_action(obs, unsquash_action=True)
        obs, r, done, _ = env.step(np.around(action))
        ep_r += r
        if done:
            break
    all_r.append(ep_r)
baseline = sum(all_r) / num_episodes
print(baseline)

/home/dibya/miniconda3/envs/real_world_deep_rl_cpu/lib/python3.9/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(
2023-08-10 20:36:08,534	WARNING deprecation.py:45 -- DeprecationWarning: `compute_action` has been deprecated. Use `Trainer.compute_single_action()` instead. This will raise an error in the future!


178504.99510324348


## Population Based Training (PBT) lives up to its promise!

- More stable training.
- Higher performance.